### Problem Statement:
Task 1: Generate datasets of functions with their Taylor expansions up the fourth order. Tokenize the dataset.

Task 2: Train an LSTM model to learn the Taylor expansion of each function.

Task 3: Similarly Train a Transformer  model to learn the Taylor expansion of each function.

In [ ]:
import re
import random
import math
import numpy as np
import torch
from sympy import *
from tqdm import tqdm
import tokenize
from io import StringIO
from torch import nn
from torch.autograd import Variable
import pandas as pd
import torch.nn.functional as F

In [ ]:
class MathExpression:
    operations = {
        'sin': 1, 'cos': 1, 'tan': 1, 'square': 1, 'cube': 1, 'exp': 1, 'log': 1,
        '+': 2, '-': 2, '*': 2, '/': 2, '**': 2
    }

    infix_notation = {
        'sin': lambda a: f'sin({a[0]})',
        'cos': lambda a: f'cos({a[0]})',
        'tan': lambda a: f'tan({a[0]})',
        'square': lambda a: f'({a[0]})**2',
        'cube': lambda a: f'({a[0]})**3',
        'exp': lambda a: f'exp({a[0]})',
        'log': lambda a: f'log({a[0]})',
        '+': lambda a: f'({a[0]})+({a[1]})',
        '-': lambda a: f'({a[0]})-({a[1]})',
        '*': lambda a: f'({a[0]})*({a[1]})',
        '/': lambda a: f'({a[0]})/({a[1]})',
        '**': lambda a: f'({a[0]})**({a[1]})'
    }

    unary_probabilities = {'sin': 1, 'cos': 1, 'tan': 2, 'square': 4, 'cube': 3, 'exp': 2, 'log': 1}
    binary_probabilities = {'+': 3, '-': 3, '*': 2, '/': 2, '**': 1}
    sympy_to_ops = {sin: 'sin', cos: 'cos', tan: 'tan', exp: 'exp', log: 'log', Add: '+', Mul: '*', Pow: '**'}

    def unary_binary_dist(self, size):
        max_e = size * 2
        D = np.zeros((max_e + 2, size + 1))
        D[:, 0] = (self._num_leaves ** np.arange(max_e + 2))
        D[0, 0] = 0

        for n in range(1, size + 1):
            for e in range(1, max_e + 1):
                D[e, n] = (self._num_leaves * D[e-1, n] +
                           self._num_unary_ops * D[e, n-1] +
                           self._num_bin_ops * D[e+1, n-1])
        return D[:max_e + 1, :size + 1]

    def sample(self, e, n):
        P = np.zeros((e, 2))
        k_vals = np.arange(e)
        P[:, 0] = (self._num_leaves ** k_vals) * self._num_unary_ops * self._unary_binary_dist[e - k_vals, n-1]
        P[:, 1] = (self._num_leaves ** k_vals) * self._num_bin_ops * self._unary_binary_dist[e - k_vals + 1, n-1]
        P_flat = P.T.flatten()
        P_flat /= P_flat.sum()
        k = np.random.choice(2 * e, p=P_flat)
        return k % e, 1 if k < e else 2

    def choose_unary_op(self):
        ops, probs = zip(*self.unary_probabilities.items())
        probs = np.array(probs) / sum(probs)
        return np.random.choice(ops, p=probs)

    def choose_bin_op(self):
        ops, probs = zip(*self.binary_probabilities.items())
        probs = np.array(probs) / sum(probs)
        return np.random.choice(ops, p=probs)

    def choose_leaf(self):
        return 'x' if random.random() < 0.3 else random.randint(0, 9)

    def gen_from_sympy(self, expr):
        self._rep = []
        stack = [expr]
        while stack:
            curr = stack.pop()
            if isinstance(curr, (Symbol, Integer)):
                self._rep.append(str(curr))
            elif isinstance(curr, Rational):
                self._rep.extend(['/', str(curr.p), str(curr.q)])
            elif curr in [E, pi, I]:
                self._rep.append(str(curr).lower())
            else:
                op = self.sympy_to_ops.get(type(curr), None)
                if op:
                    args = curr.args
                    self._rep.extend([op] * (len(args) - 1))
                    stack.extend(reversed(args))

    def gen_random(self, num_ops):
        self._num_leaves = 1
        self._num_bin_ops = len(self.binary_probabilities)
        self._num_unary_ops = len(self.unary_probabilities)
        self._unary_binary_dist = self.unary_binary_dist(num_ops + 1)

        rep = [None]
        e = 1
        skipped_total = 0
        for n in range(num_ops, 0, -1):
            k, arity = self.sample(e, n)
            skipped_total += k
            none_indices = [i for i, x in enumerate(rep) if x is None]
            pos = none_indices[skipped_total]
            op = self.choose_unary_op() if arity == 1 else self.choose_bin_op()
            new_elems = [op] + [None] * arity
            rep[pos:pos+1] = new_elems
            e = e - k + (0 if arity == 1 else 1)
            skipped_total = 0

        for i in range(len(rep)):
            if rep[i] is None:
                rep[i] = self.choose_leaf()
        self._rep = rep

    def __init__(self, expr=None, num_ops=None):
        if expr is not None:
            self.gen_from_sympy(expr)
        else:
            self.gen_random(num_ops)

    def to_infix(self):
        stack = []
        for token in reversed(self._rep):
            if token in self.operations:
                args = [stack.pop() for _ in range(self.operations[token])]
                stack.append(self.infix_notation[token](args[::-1]))
            else:
                stack.append(str(token))
        return stack[0]

    def get_rep(self):
        return self._rep

In [ ]:
def taylor_series(f_str, a, order):
    x = symbols('x')
    f = parse_expr(f_str)
    taylor = f.subs(x, a)
    for i in range(1, order + 1):
        f = diff(f, x)
        term = (f * (x - a)**i) / factorial(i)
        taylor += term
    return taylor.expand()

def gen_pair(ops=3):
    expr = MathExpression(num_ops=ops)
    tay = taylor_series(expr.to_infix(), Symbol('a'), 4)
    return expr, MathExpression(expr=tay)

In [ ]:
class FunctionDataset(torch.utils.data.Dataset):
    def __init__(self, ops=3, max_seq_length=32, num_items=100):
        self.max_seq = max_seq_length
        self.data = []
        while len(self.data) < num_items:
            src, tgt = gen_pair(ops)
            src_rep = ['<SOS>'] + src.get_rep() + ['<EOS>']
            tgt_rep = ['<SOS>'] + tgt.get_rep() + ['<EOS>']
            if len(src_rep) <= max_seq_length and len(tgt_rep) <= max_seq_length:
                self.data.append((src_rep, tgt_rep))

        all_tokens = set(token for pair in self.data for expr in pair for token in expr)
        self.vocab = {'<PAD>': 0}
        self.vocab.update({token: i+1 for i, token in enumerate(all_tokens)})

        self.inputs = []
        self.targets = []
        for src, tgt in self.data:
            src_ids = [self.vocab.get(t, 0) for t in src] + [0] * (max_seq_length - len(src))
            tgt_ids = [self.vocab.get(t, 0) for t in tgt] + [0] * (max_seq_length - len(tgt))
            self.inputs.append(torch.LongTensor(src_ids))
            self.targets.append(torch.LongTensor(tgt_ids))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

    def get_vocab(self):
        return self.vocab
    def get_alphabet(self):
        return self.vocab

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
d = FunctionDataset(num_items=500)
train_size = int(0.9 * len(d))
train_dataset, test_dataset = torch.utils.data.random_split(d, [train_size, len(d)-train_size])

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim=512, num_layers=2, hidden_size=512, dropout=0.2):
    super(Encoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.num_layers = num_layers
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(
        num_embeddings=vocab_size,
        embedding_dim=self.embedding_dim
    )
    self.lstm = nn.LSTM(
        input_size=self.embedding_dim,
        hidden_size=self.hidden_size,
        num_layers=self.num_layers,
        dropout=dropout,
    )

  def forward(self, x):
    embed = self.embedding(x)
    output, (h,c) = self.lstm(embed)
    return h, c

class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim=512, num_layers=2, hidden_size=512, dropout=0.2):
    super(Decoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.num_layers = num_layers
    self.output_size = vocab_size
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(
        num_embeddings=vocab_size,
        embedding_dim=self.embedding_dim
    )
    self.lstm = nn.LSTM(
        input_size=self.embedding_dim,
        hidden_size=self.hidden_size,
        num_layers=self.num_layers,
        dropout=0.2,
    )
    self.out = nn.Linear(self.hidden_size, self.output_size)
    self.softmax = nn.LogSoftmax(dim=2)
    self.to(device)

  def forward(self, input, h_0, c_0):
    embedded = self.embedding(input.unsqueeze(0))
    output, (h,c) = self.lstm(embedded, (h_0, c_0))
    output = self.out(output)
    output = self.softmax(output)
    return output.squeeze(0), h , c

In [ ]:
class Model(nn.Module):
  def __init__(self, encoder, decoder):
    super(Model, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.to(device)

  def forward(self, input, tgt=None):
    if len(input.shape) < 2:
        input = input.unsqueeze(1)
    batch_size = input.shape[1]
    h, c = enc(input)
    target = torch.zeros(batch_size, dtype=torch.long).to(device)
    if tgt is None:
      max_seq_length = input.shape[0]
      target[:] = d.token_to_idx['<SOS>']
    else:
      max_seq_length = tgt.shape[1]
      target[:] = tgt[:,0]
    outputs = torch.zeros(max_seq_length, batch_size, dec.output_size, dtype=torch.float).to(device)
    for i in range(max_seq_length):
        prediction, h, c = dec(target, h, c)
        outputs[i] = prediction
        if tgt is None:
          target = prediction.argmax(dim=1)
        else:
          target = tgt[:,i]
    return outputs

In [ ]:
enc = Encoder(len(d.get_alphabet()) + 1)
dec = Decoder(len(d.get_alphabet()) + 1)
m = Model(enc,dec).to(device)

In [ ]:
def test_epoch_LSTM(model, test_loader, criterion, batch_size=4):
  model.eval()
  total_loss = 0
  total_items = 0
  num_correct = 0
  for src, tgt in tqdm(test_loader):
    src = src.to(device)
    tgt = tgt.to(device)

    pred = model(src.squeeze().T, tgt=tgt[:,:-1])
    pred = pred.permute((1,2,0))
    tgt_out = tgt[:,1:]

    loss = criterion(pred, tgt_out)

    total_loss += loss.item()
    total_items += (tgt_out != 0).sum(dim=(0,1))

    num_correct += (torch.logical_and((logits.argmax(dim=2) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss, num_correct / total_items

def train_epoch_LSTM(model, train_loader, optimizer, criterion, batch_size=4):
  model.train()
  total_loss = 0
  total_items = 0
  num_correct = 0
  for src, tgt in tqdm(train_loader):
    src = src.to(device)
    tgt = tgt.to(device)

    pred = model(src.squeeze().T,tgt=tgt[:,:-1])

    pred = pred.permute((1,2,0))
    tgt_out = tgt[:,1:]
    loss = criterion(pred, tgt_out)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_items += (tgt_out != 0).sum(dim=(0,1))

    num_correct += (torch.logical_and((pred.argmax(dim=1) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss, num_correct / total_items

def train_LSTM(model, train_dataset,  test_dataset, batch_size=32, epochs=100):
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
  criterion = nn.CrossEntropyLoss()


  optim = torch.optim.Adam(model.parameters(), lr=1e-3)
  for e in range(epochs):
    train_loss, train_acc = train_epoch_LSTM(model, train_loader, optim, criterion, batch_size=batch_size)
    test_loss, test_acc = train_epoch_LSTM(model, test_loader, optim, criterion, batch_size=batch_size)
    print(f'Epoch: {e + 1} Training Loss: {train_loss} Training Accuracy: {train_acc} Test Loss: {test_loss} Test Accuracy: {test_acc}')

In [ ]:
train_LSTM(m, train_dataset, test_dataset, batch_size=32)

100%|██████████| 2/2 [00:00<00:00, 16.12it/s]


Epoch: 1 Training Loss: 23.17210751771927 Training Accuracy: 0.05978068709373474 Test Loss: 1.4430557489395142 Test Accuracy: 0.0917721539735794


100%|██████████| 2/2 [00:00<00:00, 17.24it/s]


Epoch: 2 Training Loss: 9.684578090906143 Training Accuracy: 0.16413158178329468 Test Loss: 1.2323780059814453 Test Accuracy: 0.2278480976819992


100%|██████████| 2/2 [00:00<00:00, 17.55it/s]


...

Epoch: 98 Training Loss: 0.01880612422246486 Training Accuracy: 1.0 Test Loss: 0.0023057444486767054 Test Accuracy: 1.0


100%|██████████| 2/2 [00:00<00:00, 17.25it/s]


Epoch: 99 Training Loss: 0.017812869395129383 Training Accuracy: 1.0 Test Loss: 0.00205195602029562 Test Accuracy: 1.0


100%|██████████| 2/2 [00:00<00:00, 16.57it/s]

Epoch: 100 Training Loss: 0.01802925457013771 Training Accuracy: 1.0 Test Loss: 0.002137118368409574 Test Accuracy: 1.0


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

  src_padding_mask = (src == 0).transpose(0, 1)
  tgt_padding_mask = (tgt == 0).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, num_encoder_layers, nhead, num_decoder_layers,
                 emb_size, src_vocab_size, tgt_vocab_size,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(TransformerModel, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_size, nhead=nhead,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = nn.TransformerDecoderLayer(d_model=emb_size, nhead=nhead,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.emb_size = emb_size
        self.src_tok_emb = self.embedding = nn.Embedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = self.embedding = nn.Embedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src, trg, src_mask,
                tgt_mask, src_padding_mask,
                tgt_padding_mask, memory_key_padding_mask):
        src_emb = self.positional_encoding(self.src_tok_emb(src)* math.sqrt(self.emb_size))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg)* math.sqrt(self.emb_size))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

In [ ]:
model = TransformerModel(num_encoder_layers=6, nhead=8, num_decoder_layers=6,
                 emb_size=512, src_vocab_size=(len(d.get_alphabet()) + 1), tgt_vocab_size=(len(d.get_alphabet()) + 1),
                 dim_feedforward = 512, dropout = 0.2).to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
def train_epoch_transformer(model, train_loader, optimizer, criterion, batch_size):
  model.train()
  total_loss = 0
  num_correct = 0
  total_items = 0
  for src, tgt in tqdm(train_loader):
      src = src.to(device).T
      tgt = tgt.to(device).T

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:,:]
      loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))

      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      total_items += (tgt_out != 0).sum(dim=(0,1))

      num_correct += (torch.logical_and((logits.argmax(dim=2) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss / len(train_loader), num_correct / total_items

def test_epoch_transformer(model, test_loader, criterion, batch_size):
  model.eval()
  total_loss = 0
  num_correct = 0
  total_items = 0
  for src, tgt in tqdm(train_loader):
      src = src.to(device).T
      tgt = tgt.to(device).T

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)


      tgt_out = tgt[1:,:]
      loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))


      total_loss += loss.item()
      total_items += (tgt_out != 0).sum(dim=(0,1))

      num_correct += (torch.logical_and((logits.argmax(dim=2) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss / len(train_loader), num_correct / total_items

In [ ]:
def train_transformer(model, train_dataset, test_dataset, batch_size=32, epochs=100):
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
  criterion = nn.CrossEntropyLoss()

  optim = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

  for e in range(epochs):
    train_loss, train_acc = train_epoch_transformer(model, train_loader, optim, criterion, batch_size=batch_size)
    test_loss, test_acc = train_epoch_transformer(model, test_loader, optim, criterion, batch_size=batch_size)
    print(f'Epoch: {e + 1} Training Loss: {train_loss} Training Accuracy: {train_acc} Test Loss: {test_loss} Test Accuracy: {test_acc}')

In [ ]:
train_transformer(model,train_dataset, test_dataset)

  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 2/2 [00:00<00:00, 17.12it/s]


Epoch: 1 Training Loss: 1.7636406501134236 Training Accuracy: 0.0003537318843882531 Test Loss: 1.1341528594493866 Test Accuracy: 0.0


100%|██████████| 2/2 [00:00<00:00, 17.32it/s]


Epoch: 2 Training Loss: 1.1200006524721782 Training Accuracy: 0.0 Test Loss: 0.9625139534473419 Test Accuracy: 0.0


100%|██████████| 2/2 [00:00<00:00, 16.90it/s]


...

Epoch: 98 Training Loss: 0.02349249434967836 Training Accuracy: 0.9699327945709229 Test Loss: 0.021861808374524117 Test Accuracy: 0.9683544039726257


100%|██████████| 2/2 [00:00<00:00, 17.27it/s]


Epoch: 99 Training Loss: 0.01900718950976928 Training Accuracy: 0.9784223437309265 Test Loss: 0.016292295651510358 Test Accuracy: 0.9810126423835754


100%|██████████| 2/2 [00:00<00:00, 17.53it/s]

Epoch: 100 Training Loss: 0.01901131837318341 Training Accuracy: 0.9787760972976685 Test Loss: 0.01650015451014042 Test Accuracy: 0.9715189933776855
